In [5]:
!pip install -q langchain transformers tqdm

In [3]:
%reload_ext autoreload
%autoreload 2
%reload_ext dotenv
%dotenv

In [7]:
import datasets
from tqdm.auto import tqdm
from langchain.docstore.document import Document as LangchainDocument



RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(datasets.load_dataset("m-ric/huggingface_doc", split="train"))
]


  0%|          | 0/2647 [00:00<?, ?it/s]

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

dataset_collection = []
limit = 10
for chunk_size in [200, 400, 600]:
    for chunk_overlap in [0, int(chunk_size / 10), int(chunk_size / 20)]:
        settings = f"chunk_size:{chunk_size}-chunk_overlap:{chunk_overlap}"
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
                model_name="gpt-4",
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                add_start_index=True,
                strip_whitespace=True,
                separators=["\n\n", "\n", ".", " ", ""],
                disallowed_special=[],
                allowed_special="all"
            )
        print(f"Generating with settings {settings}")
        dataset = {
            "chunk_size": chunk_size, 
            "chunk_overlap": chunk_overlap,
            "limit": limit,
            "texts": [doc.page_content for doc in tqdm(
            text_splitter.split_documents(RAW_KNOWLEDGE_BASE[:limit]))]
        }
        dataset_collection.append(dataset)
        
json.dump(dataset_collection, open("./output/huggingface_doc_splits.json", "w"))



Generating with settings chunk_size:200-chunk_overlap:0


  0%|          | 0/91 [00:00<?, ?it/s]

Generating with settings chunk_size:200-chunk_overlap:20


  0%|          | 0/93 [00:00<?, ?it/s]

Generating with settings chunk_size:200-chunk_overlap:10


...

  0%|          | 0/29 [00:00<?, ?it/s]

Generating with settings chunk_size:600-chunk_overlap:60


  0%|          | 0/29 [00:00<?, ?it/s]

Generating with settings chunk_size:600-chunk_overlap:30


  0%|          | 0/29 [00:00<?, ?it/s]